In [1]:
# https://dplyr.tidyverse.org/reference/context.html
%run nb_helpers.py
from datar.all import *

nb_header(cur_group_id, cur_group_rows, cur_data, cur_data_all, cur_column, book='context')

### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_group_id</div>

##### Get the current group id

##### Args:
&emsp;&emsp;`_data`: The dataframe  

##### Returns:
&emsp;&emsp;The current group id  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_group_rows</div>

##### Get the current group row indices

##### Args:
&emsp;&emsp;`_data`: The dataframe  

##### Returns:
&emsp;&emsp;The current group rows  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_data</div>

##### Get the current dataframe

##### Args:
&emsp;&emsp;`_data`: The dataframe  

##### Returns:
&emsp;&emsp;The current dataframe  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_data_all</div>

##### Get the current data for the current group including
the grouping variables  

##### Args:
&emsp;&emsp;`_data`: The dataframe  

##### Returns:
&emsp;&emsp;The current dataframe  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_column</div>

##### Get the current column

##### Args:
&emsp;&emsp;`_data`: The dataframe  
&emsp;&emsp;`_name`: The column name  

##### Returns:
&emsp;&emsp;The current column  


In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

,g,n
,<object>,<int64>
0,a,1
1,b,2
2,c,3


In [3]:
gf

,g,x,y
,<object>,<float64>,<float64>
0,a,0.879733,0.242456
1,b,0.789550,0.165711
2,b,0.073877,0.198040
3,c,0.677826,0.186310
4,c,0.324069,0.212226
5,c,0.589881,0.990174


In [4]:
gf >> mutate(id=cur_group_id()) 

,g,x,y,id
,<object>,<float64>,<float64>,<int64>
0,a,0.879733,0.242456,0
1,b,0.789550,0.165711,1
2,b,0.073877,0.198040,1
3,c,0.677826,0.186310,2
4,c,0.324069,0.212226,2
5,c,0.589881,0.990174,2


In [5]:
gf >> summarise(row=cur_group_rows()) 

,g,row
,<object>,<object>
0,a,[0]
1,b,"[1, 2]"
2,c,"[3, 4, 5]"


In [6]:
gf_group = gf >> summarise(data=cur_group())
gf_group 

,g,data
,<object>,<object>
0,a,<DF 1x1>
1,b,<DF 1x1>
2,c,<DF 1x1>


In [7]:
gf_group >> pull(f.data)

0    <DF 1x1>
1    <DF 1x1>
2    <DF 1x1>
Name: data, dtype: object

In [8]:
gf_data = gf >> summarise(data=cur_data())
gf_data

,g,data
,<object>,<object>
0,a,<DF 1x2>
1,b,<DF 2x2>
2,c,<DF 3x2>


In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
   <float64> <float64>
 0  0.879733  0.242456,
           x         y
   <float64> <float64>
 1  0.789550  0.165711
 2  0.073877  0.198040,
           x         y
   <float64> <float64>
 3  0.677826  0.186310
 4  0.324069  0.212226
 5  0.589881  0.990174]

In [10]:
gf_data_all = gf >> summarise(data=cur_data_all())
gf_data_all

,g,data
,<object>,<object>
0,a,<DF 1x3>
1,b,<DF 2x3>
2,c,<DF 3x3>


In [11]:
gf_data_all >> pull(f.data, to='list')

[         g         x         y
   <object> <float64> <float64>
 0        a  0.879733  0.242456,
          g         x         y
   <object> <float64> <float64>
 1        b  0.789550  0.165711
 2        b  0.073877  0.198040,
          g         x         y
   <object> <float64> <float64>
 3        c  0.677826  0.186310
 4        c  0.324069  0.212226
 5        c  0.589881  0.990174]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

,x,y
,<object>,<object>
0,x 0.7739296633011361,y 0.05878489331508395
1,x 0.6233885082054422,y 0.027460112154048803
2,x 0.005457753705443728,y 0.03921965587769912
3,x 0.45944873370090106,y 0.034711398724083777
4,x 0.10502100613889181,y 0.04504004423820979
5,x 0.3479600358358678,y 0.9804449881028017


In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

,g,x,y
,<object>,<object>,<object>
0,a,x 0.7739296633011361,y 0.05878489331508395
1,b,x 0.6233885082054422,y 0.027460112154048803
2,b,x 0.005457753705443728,y 0.03921965587769912
3,c,x 0.45944873370090106,y 0.034711398724083777
4,c,x 0.10502100613889181,y 0.04504004423820979
5,c,x 0.3479600358358678,y 0.9804449881028017
